In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import random
import shutil
from tqdm.auto import tqdm

d:\Desktop\StudiesStuff\ComputerVision\cv final project 2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
IMAGE_FOLDER = "dataset/TUSimpleProcessed/images"
MASKS_FOLDER = "dataset/TUSimpleProcessed/masks"

CLIPS_PATH = "dataset/TUSimple/train_set/clips"

os.makedirs(IMAGE_FOLDER, exist_ok=True)
os.makedirs(MASKS_FOLDER, exist_ok=True)

for clip_dir in os.listdir(CLIPS_PATH):
    clip_dir_path = os.path.join(CLIPS_PATH, clip_dir)

    print("Processing Clip:", clip_dir)

    for frame_dir in os.listdir(clip_dir_path):
        frame_path = os.path.join(clip_dir_path, frame_dir, '20.jpg')
        
        if not os.path.isfile(frame_path):
            continue

        # Use os.sep or os.path.normpath for cross-platform compatibility
        new_filename = f"{clip_dir}_{frame_dir}.jpg"
        new_file_path = os.path.join(IMAGE_FOLDER, new_filename)

        shutil.copy(frame_path, new_file_path)

Processing Clip: 0313-1
Processing Clip: 0313-2
Processing Clip: 0531
Processing Clip: 0601


In [4]:
df1 = pd.read_json("dataset/TUSimple/train_set/label_data_0313.json", lines=True)
df2 = pd.read_json("dataset/TUSimple/train_set/label_data_0531.json", lines=True)
df3 = pd.read_json("dataset/TUSimple/train_set/label_data_0601.json", lines=True)

df = pd.concat([df1, df2, df3])
df.head()

,lanes,h_samples,raw_file
0,"[[-2, -2, -2, -2, 632, 625, 617, 609, 601, 594...","[240, 250, 260, 270, 280, 290, 300, 310, 320, ...",clips/0313-1/6040/20.jpg
1,"[[-2, -2, -2, 658, 646, 635, 623, 612, 601, 58...","[240, 250, 260, 270, 280, 290, 300, 310, 320, ...",clips/0313-1/5320/20.jpg
2,"[[-2, 570, 554, 538, 522, 505, 489, 473, 456, ...","[240, 250, 260, 270, 280, 290, 300, 310, 320, ...",clips/0313-1/23700/20.jpg
3,"[[-2, -2, -2, -2, 555, 542, 530, 517, 505, 493...","[240, 250, 260, 270, 280, 290, 300, 310, 320, ...",clips/0313-1/51660/20.jpg
4,"[[-2, -2, -2, 522, 525, 528, 531, 534, 537, 53...","[240, 250, 260, 270, 280, 290, 300, 310, 320, ...",clips/0313-1/25680/20.jpg


In [ ]:
def generate_lan_mask(row):
    mask = np.zeros((720,1280,1), dtype=np.uint8)

    h_samples = row.h_samples
    lanes = row.lanes
    raw_file = row.raw_file

    for lane in lanes:
        h_samples_filtered = [y for x, y in zip(lane, h_samples) if x != -2]
        lane_filtered = [x for x in lane if x != -2]

        lane_points = np.array(list(zip(lane_filtered, h_samples_filtered)))

        cv2.polylines(mask, [lane_points], isClosed=False, color=(255,255,255), thickness=15)
    
    normalized_path = os.path.normpath(raw_file[:-7])
    parts = normalized_path.split(os.sep)[-2:]
    mask_filename = f"{parts[0]}_{parts[1]}.jpg"
    mask_filename_path = os.path.join(MASKS_FOLDER, mask_filename)

    cv2.imwrite(mask_filename_path, mask)

for index, row in tqdm(df.iterrows(), total=len(df)):
    generate_lan_mask(row)

print("total masks ", len(os.listdir(MASKS_FOLDER)))

100%|██████████| 3626/3626 [00:09<00:00, 363.42it/s]

total masks  3626
